In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
import json
import ast
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [5]:
movies=movies.merge(credits,on='title')

In [6]:
movies.shape

(4809, 23)

In [7]:
movies['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [8]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [9]:
Pred_movie=movies[['movie_id','title','overview','genres','keywords','cast','crew']].copy()

In [10]:
Pred_movie.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [11]:
Pred_movie.isnull().sum()


movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [12]:
Pred_movie.dropna(inplace=True)

In [13]:
Pred_movie.isnull().sum()


movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [14]:
Pred_movie['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [15]:
aloo=[]
for i in Pred_movie['genres']:
    aloo2=[]
    for j in json.loads(i):
        aloo2.append(j['name'])
    aloo.append(aloo2)    


In [16]:
Pred_movie['genres']=aloo

In [17]:
Pred_movie.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [18]:
aloo=[]
for i in Pred_movie['keywords']:
    aloo2=[]
    for j in json.loads(i):
        aloo2.append(j['name'])
    aloo.append(aloo2) 
Pred_movie['keywords']=aloo

In [19]:
Pred_movie.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [20]:
def conv(obj):
    l=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter !=3:
            l.append(i['character'])
            counter+=1
        else:
            break
    return l;    




In [21]:
Pred_movie['cast']=Pred_movie['cast'].apply(conv)

In [22]:
def conv2(obj):
    l=[]
    counter=0
    for i in ast.literal_eval(obj):
        if i['job'] =='Director':
            l.append(i['name'])
            break
        
    return l;    




In [23]:
Pred_movie['crew']=Pred_movie['crew'].apply(conv2)

In [24]:
Pred_movie.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Jake Sully, Neytiri, Dr. Grace Augustine]",[James Cameron]


In [25]:
Pred_movie['cast']=Pred_movie['cast'].apply(lambda x:[i.replace(" ","") for i in x])
Pred_movie['genres']=Pred_movie['genres'].apply(lambda x:[i.replace(" ","") for i in x])
Pred_movie['crew']=Pred_movie['crew'].apply(lambda x:[i.replace(" ","") for i in x])
Pred_movie['keywords']=Pred_movie['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [26]:
Pred_movie.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[JakeSully, Neytiri, Dr.GraceAugustine]",[JamesCameron]


In [27]:
Pred_movie['overview']=[x.split(' ') for x in Pred_movie['overview']]

In [28]:
Pred_movie.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[JakeSully, Neytiri, Dr.GraceAugustine]",[JamesCameron]


In [29]:
Pred_movie['tags']=Pred_movie['overview']+Pred_movie['genres']+Pred_movie['keywords']+Pred_movie['cast']+Pred_movie['crew']

In [30]:
Pred_movie.head(1)

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[JakeSully, Neytiri, Dr.GraceAugustine]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [31]:
new_movie=Pred_movie[['movie_id','title','tags']].copy()

In [32]:
new_movie

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [33]:
new_movie['tags'] = [' '.join(x).lower() for x in new_movie['tags']]


In [34]:

ps = PorterStemmer()

def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)    



In [35]:
new_movie['tags']=new_movie['tags'].apply(stem)

In [36]:

cv = CountVectorizer(max_features=5000, stop_words='english')


In [37]:
cv.fit_transform(new_movie['tags']).toarray().shape

(4806, 5000)

In [38]:
vectors=cv.fit_transform(new_movie['tags']).toarray()

In [39]:
cv.get_feature_names_out()


array(['000', '007', '10', ..., 'zombies', 'zone', 'zoo'], dtype=object)

In [40]:
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
similarities=cosine_similarity(vectors)

In [42]:
similarities

array([[1.        , 0.0862796 , 0.0862796 , ..., 0.04441156, 0.        ,
        0.        ],
       [0.0862796 , 1.        , 0.06451613, ..., 0.02490677, 0.        ,
        0.0277137 ],
       [0.0862796 , 0.06451613, 1.        , ..., 0.02490677, 0.        ,
        0.        ],
       ...,
       [0.04441156, 0.02490677, 0.02490677, ..., 1.        , 0.03774257,
        0.04279605],
       [0.        , 0.        , 0.        , ..., 0.03774257, 1.        ,
        0.08399211],
       [0.        , 0.0277137 , 0.        , ..., 0.04279605, 0.08399211,
        1.        ]])

In [43]:
mov_rec=[]
def recommend(name):
    index=new_movie[new_movie['title']==name].index[0]
    rec = sorted(enumerate(similarities[index]),key=lambda x:x[1], reverse=True)[1:6]
    for i in rec:
        mov_rec.append((new_movie.iloc[i[0]].movie_id,new_movie.iloc[i[0]].title))
    return mov_rec    
            

In [44]:
recommend('Avatar')

[(440, 'Aliens vs Predator: Requiem'),
 (679, 'Aliens'),
 (602, 'Independence Day'),
 (7450, 'Titan A.E.'),
 (76757, 'Jupiter Ascending')]

In [45]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [46]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [47]:
movies['genres']=Pred_movie['genres']
movies['cast']=Pred_movie['cast']
movies['crew']=Pred_movie['crew']
movies['keywords']=Pred_movie['keywords']




In [48]:
aloo=[]
for i in movies['spoken_languages']:
    aloo2=[]
    for j in json.loads(i):
        aloo2.append(j['name'])
    aloo.append(aloo2) 

movies['spoken_languages']=aloo

In [49]:
aloo=[]
for i in movies['production_companies']:
    aloo2=[]
    for j in json.loads(i):
        aloo2.append(j['name'])
    aloo.append(aloo2) 

movies['production_companies']=aloo

In [50]:
aloo=[]
for i in movies['production_countries']:
    aloo2=[]
    for j in json.loads(i):
        aloo2.append(j['name'])
    aloo.append(aloo2) 

movies['production_countries']=aloo

In [51]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[Action, Adventure, Fantasy, ScienceFiction]",http://www.avatarmovie.com/,19995,"[cultureclash, future, spacewar, spacecolony, ...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...",...,162.0,"[English, Español]",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[JakeSully, Neytiri, Dr.GraceAugustine]",[JamesCameron]


In [52]:
# movies.to_csv('movies.csv')

In [53]:
class MoviePredictorModel:
    def compile(self, new_movie):
        self.new_movie = new_movie

    def fit(self, similarities):
        self.similarities = similarities

    def predict(self, name,num):
        mov_rec = []
        index = self.new_movie[self.new_movie['title'].str.lower() == name.lower()].index[0]
        rec = sorted(enumerate(self.similarities[index]), key=lambda x: x[1], reverse=True)[1:num+1]
        for i in rec:
            mov_rec.append((self.new_movie.iloc[i[0]].movie_id, self.new_movie.iloc[i[0]].title))
        return mov_rec
 
        
    

In [54]:
model=MoviePredictorModel()

In [55]:
model.compile(new_movie)

In [56]:
model.fit(similarities)

In [57]:
model.predict('avatar',5)

[(440, 'Aliens vs Predator: Requiem'),
 (679, 'Aliens'),
 (602, 'Independence Day'),
 (7450, 'Titan A.E.'),
 (76757, 'Jupiter Ascending')]

In [58]:
import pickle

In [59]:

# with open('similarities.pkl','wb') as f:
#     pickle.dump(similarities,f)

In [60]:
# with open('new_movie.pkl','wb') as f:
#     pickle.dump(new_movie,f)

In [61]:
!pip install joblib
import joblib

joblib.dump(similarities, 'similarities.joblib')


['similarities.joblib']